In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext memory_profiler
import numpy as np
import pandas as pd
import sklearn

In [ ]:
from predict_sales import logger

In [ ]:
from predict_sales.data import Data

Read data and generate features.

In [ ]:
# %%mprun -f Data.process_input
Data.process_input("../input/store.csv", "../input/train.csv", "../input/test.csv", "../output/data.h5")

In [ ]:
data = pd.HDFStore('../output/data.h5')
output = pd.HDFStore('../output/output.h5')

In [ ]:
# data.close()

In [ ]:
# output.close()

Run glm predictions

In [ ]:
from predict_sales.playground import glm_predictions, xgb_predictions, mix_models

In [ ]:
# %%mprun -f glm_predictions 
glm_predictions(data, output)

Run xgboost predictions

In [ ]:
# %%mprun -f xgb_predictions 
xgb_predictions(data, output)

The final predictions are also saved in the file `mix.csv`

In [ ]:
preds = mix_models(output)

In [ ]:
preds.head()

In [ ]:
output = pd.HDFStore('../output/full_run/output.h5')

In [ ]:
glm_preds = output.get('glm_predictions').rename(columns={'PredictedSales':'Sales'})
glm_preds['Sales'] *= 0.97
glm_preds.to_csv('glm.csv', index=False)

In [ ]:
xgb_preds = output.get('xgb_predictions').rename(columns={'PredictedSales':'Sales'})
xgb_preds.to_csv('xgb.csv', index=False)

In [ ]:
preds[preds['Sales'] > 0].describe()

In [ ]:
glm_preds[glm_preds['Sales'] > 0].describe()

In [ ]:
xgb_preds[xgb_preds['Sales'] > 0].describe()

In [ ]:
joined = pd.merge(glm_preds, xgb_preds, on='Id')

In [ ]:
joined['Sales'] = (0.97 * joined['Sales_x'] + 0.985 * joined['Sales_y']) / 2

In [ ]:
joined[['Id', 'Sales']].to_csv('mix.csv', index=False)

In [ ]:
xgb_preds = pd.read_csv('xgb.csv')
xgb_preds['Sales'] *= 0.985

In [ ]:
joined = pd.merge(xgb_preds, glm_preds, on='Id')

In [ ]:
joined['Sales'] = (joined['Sales_x'] + joined['Sales_y']) / 2

In [ ]:
joined[['Id', 'Sales']].to_csv('mix.csv', index=False)